In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

test1=pd.read_csv("/Users/andrewbartnik/Desktop/MEDS/Winter/Machine Learning/Labs/ML/test.csv")
test = test1.drop(columns=test1.columns[0]).rename(columns={'TA1': 'TA1.x'})

train=pd.read_csv("/Users/andrewbartnik/Desktop/MEDS/Winter/Machine Learning/Labs/ML/train.csv")
train = train.drop(columns=[train.columns[0], train.columns[12]])

In [2]:
%whos

Variable                    Type         Data/Info
--------------------------------------------------
GradientBoostingRegressor   ABCMeta      <class 'sklearn.ensemble.<...>adientBoostingRegressor'>
GridSearchCV                ABCMeta      <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
Pipeline                    ABCMeta      <class 'sklearn.pipeline.Pipeline'>
StandardScaler              type         <class 'sklearn.preproces<...>ng._data.StandardScaler'>
StratifiedKFold             ABCMeta      <class 'sklearn.model_sel<...>._split.StratifiedKFold'>
XGBRegressor                type         <class 'xgboost.sklearn.XGBRegressor'>
mean_squared_error          function     <function mean_squared_error at 0x7f9265614940>
np                          module       <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
os                          module       <module 'os' from '/Users<...>env/lib/python3.8/os.py'>
pd                          module       <module 'pandas' from '/U<...

In [3]:
print(test.head())
train.head()

     Lat_Dec     Lon_Dec  NO2uM  NO3uM  NH3uM  R_TEMP  R_Depth  R_Sal  \
0  34.321666 -120.811666   0.02   24.0   0.41    9.51      101  189.9   
1  34.275000 -120.033333   0.00   25.1   0.00    9.84      102  185.2   
2  34.275000 -120.033333   0.00   31.9   0.00    6.60      514  124.1   
3  33.828333 -118.625000   0.00    0.0   0.20   19.21        1  408.1   
4  33.828333 -118.625000   0.02   19.7   0.00   10.65      100  215.5   

   R_DYNHT  R_Nuts  R_Oxy_micromol.Kg  PO4uM  SiO3uM    TA1.x  Salinity1  \
0    0.258    0.41         138.838300   1.85    25.5  2244.94     33.830   
1    0.264    0.00         102.709200   2.06    28.3  2253.27     33.963   
2    0.874    0.00           2.174548   3.40    88.1  2316.95     34.241   
3    0.004    0.20         258.674300   0.27     2.5  2240.49     33.465   
4    0.274    0.00         145.839900   1.64    19.4  2238.30     33.720   

   Temperature_degC  
0              9.52  
1              9.85  
2              6.65  
3             19

,Lat_Dec,Lon_Dec,NO2uM,NO3uM,NH3uM,R_TEMP,R_Depth,R_Sal,R_DYNHT,R_Nuts,R_Oxy_micromol.Kg,PO4uM,SiO3uM,TA1.x,Salinity1,Temperature_degC,DIC
0,34.385030,-120.665530,0.030,33.80,0.00,7.79,323,141.2,0.642,0.00,37.40948,2.77,53.86,2287.45,34.198,7.82,2270.17
1,31.418333,-121.998333,0.000,34.70,0.00,7.12,323,140.8,0.767,0.00,64.81441,2.57,52.50,2279.10,34.074,7.15,2254.10
2,34.385030,-120.665530,0.180,14.20,0.00,11.68,50,246.8,0.144,0.00,180.29150,1.29,13.01,2230.80,33.537,11.68,2111.04
3,33.482580,-122.533070,0.013,29.67,0.01,8.33,232,158.5,0.562,0.01,89.62595,2.27,38.98,2265.85,34.048,8.36,2223.41
4,31.414320,-121.997670,0.000,33.10,0.05,7.53,323,143.4,0.740,0.05,60.03062,2.53,49.28,2278.49,34.117,7.57,2252.62


In [4]:


X_train = train.drop(columns=['DIC'])
y_train = train['DIC']

# Split the train DataFrame into a training set and a holdout set (80% train, 20% holdout)
X_train_cv, X_holdout, y_train_cv, y_holdout = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)



In [5]:
%whos

Variable                    Type         Data/Info
--------------------------------------------------
GradientBoostingRegressor   ABCMeta      <class 'sklearn.ensemble.<...>adientBoostingRegressor'>
GridSearchCV                ABCMeta      <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
Pipeline                    ABCMeta      <class 'sklearn.pipeline.Pipeline'>
StandardScaler              type         <class 'sklearn.preproces<...>ng._data.StandardScaler'>
StratifiedKFold             ABCMeta      <class 'sklearn.model_sel<...>._split.StratifiedKFold'>
XGBRegressor                type         <class 'xgboost.sklearn.XGBRegressor'>
X_holdout                   DataFrame            Lat_Dec     Lon_D<...>\n[291 rows x 16 columns]
X_train                     DataFrame            Lat_Dec     Lon_D<...>n[1454 rows x 16 columns]
X_train_cv                  DataFrame            Lat_Dec     Lon_D<...>n[1163 rows x 16 columns]
mean_squared_error          function     <function mean_square

In [6]:


# Initialize the XGBRegressor
xgb = XGBRegressor()

# Create a pipeline with StandardScaler and the initialized XGBRegressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb)
])


In [7]:
# First, tune the learning rate
learning_rates = np.linspace(0.01, 0.35, 70)
param_grid_learning_rate = {
    'xgb__learning_rate': learning_rates
}

grid_search_lr = GridSearchCV(
    pipeline, param_grid_learning_rate, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, groups=bins
)

NameError: name 'bins' is not defined

In [ ]:
# First, tune the learning rate
learning_rates = np.linspace(0.01, 0.6, 120)
param_grid_learning_rate = {
    'xgb__learning_rate': learning_rates
}


grid_search_lr = GridSearchCV(
    pipeline, param_grid_learning_rate, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1
)
grid_search_lr.fit(X_train, y_train)
best_learning_rate = grid_search_lr.best_params_['xgb__learning_rate']
print(f"Best learning rate: {best_learning_rate}")

In [ ]:
# Tree-specific parameters
param_grid_tree = {
    'xgb__min_child_weight': range(1, 10, 1),
    'xgb__max_depth': range(1, 10, 1),
    'xgb__gamma': np.linspace(0, 1, 11),
    'xgb__n_estimators': [100, 200, 300, 400, 500]  # adjust the number of trees
}

# Use the best learning rate found in the previous search
pipeline.set_params(xgb__learning_rate=best_learning_rate)

# Perform GridSearchCV for tree-specific parameters
grid_search_tree = GridSearchCV(
    pipeline, param_grid_tree, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1
)
grid_search_tree.fit(X_train, y_train)

# Get the best tree-specific parameters
best_min_child_weight = grid_search_tree.best_params_['xgb__min_child_weight']
best_max_depth = grid_search_tree.best_params_['xgb__max_depth']
best_gamma = grid_search_tree.best_params_['xgb__gamma']
best_n_estimators = grid_search_tree.best_params_['xgb__n_estimators']

print(f"Best min_child_weight: {best_min_child_weight}")
print(f"Best max_depth: {best_max_depth}")
print(f"Best gamma: {best_gamma}")
print(f"Best n_estimators: {best_n_estimators}")


In [ ]:
# Stochastic parameters
param_grid_stochastic = {
    'xgb__subsample': np.linspace(0.5, 1, 6),
    'xgb__colsample_bytree': np.linspace(0.5, 1, 6),
}

# Use the best learning rate and tree-specific parameters found in the previous searches
pipeline.set_params(
    xgb__learning_rate=best_learning_rate,
    xgb__min_child_weight=best_min_child_weight,
    xgb__max_depth=best_max_depth,
    xgb__gamma=best_gamma,
    xgb__n_estimators=best_n_estimators
)

# Perform GridSearchCV for stochastic parameters
grid_search_stochastic = GridSearchCV(
    pipeline, param_grid_stochastic, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1
)
grid_search_stochastic.fit(X_train, y_train)

# Get the best stochastic parameters
best_subsample = grid_search_stochastic.best_params_['xgb__subsample']
best_colsample_bytree = grid_search_stochastic.best_params_['xgb__colsample_bytree']

print(f"Best subsample: {best_subsample}")
print(f"Best colsample_bytree: {best_colsample_bytree}")


In [ ]:
# Train the final model with the best hyperparameters on the training set (excluding the holdout set)
final_pipeline = pipeline.set_params(
    xgb__learning_rate=best_learning_rate,
    xgb__min_child_weight=best_min_child_weight,
    xgb__max_depth=best_max_depth,
    xgb__gamma=best_gamma,
    xgb__n_estimators=best_n_estimators,
    xgb__subsample=best_subsample,
    xgb__colsample_bytree=best_colsample_bytree
)
final_pipeline.fit(X_train_cv, y_train_cv)

# Validate the final model on the holdout set
holdout_preds = final_pipeline.predict(X_holdout)
holdout_rmse = mean_squared_error(y_holdout, holdout_preds, squared=False)
print(f"Holdout RMSE: {holdout_rmse}")

# Train the final model with the best hyperparameters on the entire training data (including the holdout set)
final_pipeline.fit(X_train, y_train)

# Make predictions on the test set
test_preds = final_pipeline.predict(test)



In [ ]:

# Evaluate the tuned model on the holdout set
y_holdout_pred = grid_search.predict(X_holdout)
holdout_rmse = np.sqrt(mean_squared_error(y_holdout, y_holdout_pred))
print(f"RMSE on holdout set: {holdout_rmse}")

In [ ]:

# Make predictions for the test set
predictions = grid_search.predict(test1.drop(columns=['Index']))

# Create a new DataFrame with the same index as the test DataFrame and the predictions
test_preds = pd.DataFrame({'id': test1['Index'], 'DIC': predictions})


In [ ]:
predictions

In [ ]:
# Write the test_preds DataFrame to a CSV file named xgboost.csv
test_preds.to_csv('xgboost.csv', index=False)